# ME449 - Homework 3 - Zhengyang Kris Weng Submission

In [21]:
import numpy as np
import modern_robotics as mr
from tqdm import tqdm
import ur5_parameters
ur5 = ur5_parameters.UR5()

## Part0: Puppet function

## Part 1: Simulating a falling robot. 
In the first part, the robot will fall in gravity without
damping or the external spring (joint damping and spring stiffness are set to zero). Since there is
no damping or friction, the total energy of the robot (kinetic plus potential) should remain constant
during motion. Gravity is g = 9.81 m/s2 in the −ˆzs-direction, i.e., gravity acts downward.
Simulate the robot falling from rest at the home configuration for five seconds. The output data
should be saved as a .csv file, where each of the N rows has six numbers separated by commas.
This .csv file is suitable for animation with the CoppeliaSim UR5 csv animation scene. Adjust
the animation scene playback speed (“Time Multiplier”) so it takes roughly five seconds of wall
clock time to play your csv file. You can evaluate if your simulation is preserving total energy by
visually checking if the robot appears to swing to the same height (same potential energy) each
swing. Choose values of dt (a) where the energy appears nearly constant (without choosing dt
unnecessarily small) and (b) where the energy does not appear constant (because your timestep
is too coarse). Capture a video for each case and note the dt chosen for each case. Explain how
you would calculate the total energy of the robot at each timestep if you wanted to plot the total
energy to confirm that your simulation approximately preserves it.

In [26]:
# Puppet function:
def puppet_q1(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, restLength):
    """
    Simulate a robot under damping and spring reaction. Q1: free falling in gravity

    Args:
        thetalist (np.array): n-vector of initial joint angles (rad)
        dthetalist (np.array): n-vector of initial joint velocities (rad/s)
        g (np.array): 3-vector of gravity in s frame (m/s^2)
        Mlist (np.array): 8 frames of link configuration at home pose
        Slist (np.array): 6-vector of screw axes at home configuration
        Glist (np.array): Spatial inertia matrices of the links
        t (float): total simulation time (s)
        dt (float): simulation time step (s)
        damping (float): viscous damping coefficient (Nmn/rad)
        stiffness (float): spring stiffness coefficient (N/m)
        restLength (float): length of the spring at rest (m)
    Returns:
        thetamat (np.array): N x n matrix of joint angles (rad). Each row is a set of joint angles
        dthetamat (np.array): N x n matrix of joint velocities (rad/s). Each row is a set of joint velocities
    """
    # Initialize 
    N = int(t/dt)
    n = len(thetalist)
    thetamat = np.zeros((N + 1, n))
    dthetamat = np.zeros((N + 1, n))
    thetamat[0] = thetalist
    dthetamat[0] = dthetalist

    for i in tqdm(range(N)):
        i_acc = mr.ForwardDynamics(thetalist, dthetalist, np.zeros(n), g, np.zeros(n), Mlist, Glist, Slist) 
        i_pos, i_vel = mr.EulerStep(thetalist, dthetalist, i_acc, dt)
        thetamat[i + 1] = i_pos
        dthetamat[i + 1] = i_vel

        # Update
        thetalist = i_pos
        dthetalist = i_vel
    
    return thetamat, dthetamat

In [29]:
q1_thetalist0 = np.array([0, 0, 0, 0, 0, 0])
q1_dthetalist0 = np.array([0, 0, 0, 0, 0, 0])
g = np.array([0, 0, -9.81])

q1_thetamat, _ = puppet_q1(q1_thetalist0, q1_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.001, 0, 0, 0)

# Save to csv file
np.savetxt('q1_thetamat.csv', q1_thetamat, delimiter=',')

100%|██████████| 5000/5000 [00:34<00:00, 143.02it/s]


In [ ]:
q1_thetamat_coarse, _ = puppet_q1(q1_thetalist0, q1_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.01, 0, 0, 0)

# Save to csv file
np.savetxt('q1_thetamat_coarse.csv', q1_thetamat_coarse, delimiter=',')